In [4]:
from pathlib import Path
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

DATA_DIR = Path("../data")
SAMPLE_DIR = Path("../data_sample")
SAMPLE_DIR.mkdir(parents=True, exist_ok=True)

ACCEPTED_CSV = DATA_DIR / "accepted_2007_to_2018Q4.csv"
REJECTED_CSV = DATA_DIR / "rejected_2007_to_2018Q4.csv"

# We need to create a sample because the full dataset is too large to fit in memory
SAMPLE_PATH = SAMPLE_DIR / "stage1_sample.parquet"

RANDOM_STATE = 123
# sample of 500k should be sufficient
SAMPLE_N = 500_000


In [5]:
acc_head = pd.read_csv(ACCEPTED_CSV, nrows=5, low_memory=False)
rej_head = pd.read_csv(REJECTED_CSV, nrows=5, low_memory=False)

print("Accepted columns (first 40):")
print(acc_head.columns.tolist()[:40])

print("\nRejected columns (all):")
print(rej_head.columns.tolist())

display(acc_head.head(2))
display(rej_head.head(2))


Accepted columns (first 40):
['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv']

Rejected columns (all):
['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score', 'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length', 'Policy Code']


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0


Stage 1 has to have features that exist in both datasets. The features that aren't in accepted will be used in stage 2. 

In [ ]:
CANONICAL_COLS = [
    "loan_amount",    
    "emp_length",      
    "dti",             
    "fico_est",        
]
LABEL_COL = "is_accepted"


In [ ]:
ACC_MAP = {
    "loan_amnt": "loan_amount",
    "emp_length": "emp_length",
    "dti": "dti",
}

#rejected file mappings
REJ_MAP = {
    "amount_requested": "loan_amount",
    "employment_length": "emp_length",
    "debt_to_income_ratio": "dti",
    "risk_score": "fico_est", 
}


Functions are used for readability and debugging purposes. This function will help create the columns we will use in the sample. We will create the new fico_est by averaging the range in the accepted dataset, clean headers, and create simple numeric data for later use.

In [8]:
def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    """Make headers predictable: lowercase + underscores."""
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.lower()
        .str.replace(r"[^\w]+", "_", regex=True)
        .str.strip("_")
    )
    return df

def to_float(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors="coerce")

def pct_to_float(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.replace("%", "", regex=False).str.strip()
    return pd.to_numeric(s, errors="coerce")

def clean_emp_length(series: pd.Series) -> pd.Series:
    """
    Convert employment length strings to numeric years:
      "< 1 year" -> 0.5
      "10+ years" -> 10
      "n/a" -> NaN
      "3 years" -> 3
      "1 year" -> 1
    """
    s = series.astype(str).str.lower().str.strip()
    s = s.replace({"n/a": np.nan, "na": np.nan, "none": np.nan})
    s = s.str.replace("years", "", regex=False).str.replace("year", "", regex=False).str.strip()

    s = s.replace({"< 1": "0.5", "<1": "0.5", "10+": "10"})
    return pd.to_numeric(s, errors="coerce")

def clip_fico(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    return s.clip(lower=300, upper=850)

def summarize_missing(df: pd.DataFrame, cols: list[str]) -> pd.Series:
    return df[cols].isna().mean().sort_values(ascending=False)


In [ ]:
def build_accepted_canonical(acc_raw: pd.DataFrame) -> pd.DataFrame:
    acc = acc_raw.copy()

    out = pd.DataFrame()
    for raw_col, canon_col in ACC_MAP.items():
        if raw_col in acc.columns:
            out[canon_col] = acc[raw_col]

    #Create new fico_est from fico ranges if available
    if "fico_range_low" in acc.columns and "fico_range_high" in acc.columns:
        low = to_float(acc["fico_range_low"])
        high = to_float(acc["fico_range_high"])
        out["fico_est"] = (low + high) / 2.0
    elif "fico_range_low" in acc.columns:
        out["fico_est"] = to_float(acc["fico_range_low"])
    elif "fico_range_high" in acc.columns:
        out["fico_est"] = to_float(acc["fico_range_high"])
    else:
        out["fico_est"] = np.nan

    #Clean types
    out["loan_amount"] = to_float(out["loan_amount"])
    out["dti"] = pct_to_float(out["dti"])
    out["emp_length"] = clean_emp_length(out["emp_length"])
    out["fico_est"] = clip_fico(out["fico_est"])

    out[LABEL_COL] = 1
    return out[CANONICAL_COLS + [LABEL_COL]]


In [ ]:
def build_rejected_canonical(rej_raw: pd.DataFrame) -> pd.DataFrame:
    rej = normalize_headers(rej_raw)

    out = pd.DataFrame()
    for raw_col, canon_col in REJ_MAP.items():
        if raw_col in rej.columns:
            out[canon_col] = rej[raw_col]

    # Ensure all canonical cols exist
    for c in CANONICAL_COLS:
        if c not in out.columns:
            out[c] = np.nan

    # Clean types
    out["loan_amount"] = to_float(out["loan_amount"])
    out["dti"] = pct_to_float(out["dti"])
    out["emp_length"] = clean_emp_length(out["emp_length"])
    out["fico_est"] = clip_fico(out["fico_est"])

    out[LABEL_COL] = 0
    return out[CANONICAL_COLS + [LABEL_COL]]


In [11]:
acc_raw = pd.read_csv(ACCEPTED_CSV, low_memory=False)
rej_raw = pd.read_csv(REJECTED_CSV, low_memory=False)

acc_canon = build_accepted_canonical(acc_raw)
rej_canon = build_rejected_canonical(rej_raw)

print("Accepted canonical shape:", acc_canon.shape)
print("Rejected canonical shape:", rej_canon.shape)

display(acc_canon.head())
display(rej_canon.head())


Accepted canonical shape: (2260701, 6)
Rejected canonical shape: (27648741, 6)


,loan_amount,annual_income,emp_length,dti,fico_est,is_accepted
0,3600.0,55000.0,10.0,5.91,677.0,1
1,24700.0,65000.0,10.0,16.06,717.0,1
2,20000.0,63000.0,10.0,10.78,697.0,1
3,35000.0,110000.0,10.0,17.06,787.0,1
4,10400.0,104433.0,3.0,25.37,697.0,1


,loan_amount,annual_income,emp_length,dti,fico_est,is_accepted
0,1000.0,NaN,4.0,10.00,693.0,0
1,1000.0,NaN,0.5,10.00,703.0,0
2,11000.0,NaN,1.0,10.00,715.0,0
3,6000.0,NaN,0.5,38.64,698.0,0
4,1500.0,NaN,0.5,9.43,509.0,0


In [12]:
def fill_canonical(df: pd.DataFrame, label_value: int) -> pd.DataFrame:
    df = df.copy()

    #Fill medians separately by class to avoid mixing distributions
    for col in ["loan_amount", "annual_income", "dti", "emp_length", "fico_est"]:
        med = df[col].median(skipna=True)
        df[col] = df[col].fillna(med)

    #Remove outlier values that are clearly invalid
    df["dti"] = df["dti"].clip(lower=0, upper=80)
    df["loan_amount"] = df["loan_amount"].clip(lower=0)
    df["annual_income"] = df["annual_income"].clip(lower=0)
    df["emp_length"] = df["emp_length"].clip(lower=0, upper=50)
    df["fico_est"] = df["fico_est"].clip(lower=300, upper=850)

    df[LABEL_COL] = label_value
    return df

print("Missingness BEFORE fill (accepted):")
display(summarize_missing(acc_canon, CANONICAL_COLS))

print("Missingness BEFORE fill (rejected):")
display(summarize_missing(rej_canon, CANONICAL_COLS))

acc_clean = fill_canonical(acc_canon, 1)
rej_clean = fill_canonical(rej_canon, 0)

print("Missingness AFTER fill (accepted):")
display(summarize_missing(acc_clean, CANONICAL_COLS))

print("Missingness AFTER fill (rejected):")
display(summarize_missing(rej_clean, CANONICAL_COLS))


Missingness BEFORE fill (accepted):


emp_length       0.064998
dti              0.000771
annual_income    0.000016
loan_amount      0.000015
fico_est         0.000015
dtype: float64

Missingness BEFORE fill (rejected):


annual_income    1.000000
fico_est         0.669023
emp_length       0.034409
loan_amount      0.000000
dti              0.000000
dtype: float64

C:\Users\john_\AppData\Roaming\Python\Python314\site-packages\numpy\lib\_nanfunctions_impl.py:1213: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Missingness AFTER fill (accepted):


loan_amount      0.0
annual_income    0.0
emp_length       0.0
dti              0.0
fico_est         0.0
dtype: float64

Missingness AFTER fill (rejected):


annual_income    1.0
loan_amount      0.0
emp_length       0.0
dti              0.0
fico_est         0.0
dtype: float64

Combine the two datasets and create the stratified sample. Saving as a Parquet for efficiency

In [ ]:
df_full = pd.concat([acc_clean, rej_clean], ignore_index=True)

#stratified sampling to preserve class ratio
df_pos = df_full[df_full[LABEL_COL] == 1]
df_neg = df_full[df_full[LABEL_COL] == 0]

pos_ratio = len(df_pos) / len(df_full)
n_pos = int(SAMPLE_N * pos_ratio)
n_neg = SAMPLE_N - n_pos

df_sample = pd.concat(
    [
        df_pos.sample(n=min(n_pos, len(df_pos)), random_state=RANDOM_STATE),
        df_neg.sample(n=min(n_neg, len(df_neg)), random_state=RANDOM_STATE),
    ],
    ignore_index=True
).sample(frac=1.0, random_state=RANDOM_STATE).reset_index(drop=True)

print("Sample shape:", df_sample.shape)
print("Sample label distribution:")
print(df_sample[LABEL_COL].value_counts(normalize=True))

df_sample.to_parquet(SAMPLE_PATH, index=False)
print("✅ Saved sample to:", SAMPLE_PATH)

# Set df1 for training
df1 = df_sample
df1.head()


In [ ]:
df1 = pd.read_parquet(SAMPLE_PATH)
print(df1.shape)
df1.head()
